In [ ]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
minhas_apostas = [
    "05 15 26 27 46 53",  # aposta n.° 1
    "03 12 19 20 45 47",  # aposta n.° 2
    "01 10 17 41 42 56",  # aposta n.° 3
    "02 10 13 27 53 55",  # aposta n.° 4
    "06 07 08 11 43 56",  # aposta n.° 5
    "08 10 14 25 33 34",  # aposta n.° 6
    "05 11 16 40 43 57",  # aposta n.° 7
    "04 05 08 13 17 38",  # aposta n.° 8
    "13 24 32 49 51 60",  # aposta n.° 9
    "11 16 19 43 58 60",  # aposta n.° 10
    "03 05 10 20 35 46",  # aposta n.° 11
    "02 09 10 19 31 57",  # aposta n.° 12
    "04 18 20 21 39 57",  # aposta n.° 13
    "02 11 22 36 49 60",  # aposta n.° 14
    "02 21 39 48 52 57",  # aposta n.° 15
    "14 41 45 50 54 59",  # aposta n.° 16
    "13 20 22 25 28 39",  # aposta n.° 17
    "01 16 21 34 49 54",  # aposta n.° 18
]

In [ ]:
mega = {"concurso": [], "data": [], "bolas": [], "ganhou": []}

for row in pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine).itertuples():
    for jogo in minhas_apostas:
        if jogo in row.bolas:
            mega["concurso"].append(row.concurso)
            mega["data"].append(row.data)
            mega["bolas"].append(row.bolas)
            mega["ganhou"].append(row.ganhou)

pd.DataFrame(mega)

In [ ]:
jogos = "02 03 04 05 06 07"
print("consta") if "02" in jogos else print("não consta")
print("consta") if "08" in jogos else print("não consta")

In [ ]:
mega = {k: 0 for k in range(1, 61)}

for x in minhas_apostas:
    y = x.split()
    for z in range(len(y)):
        mega[int(y[z])] += 1

print(mega)

In [ ]:
mega = {}

for x in minhas_apostas:
    y = x.split()
    for z in range(len(y)):
        try:
            mega[int(y[z])] += 1
        except KeyError:
            mega[int(y[z])] = 1

print(mega)

In [ ]:
new = [
    {"concurso": 2747, "data": "2024-07-10", "bolas": "14 17 24 28 36 45", "ganhou": False},
]

df_new = pd.DataFrame(new)

rows_inserted = df_new.to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {rows_inserted} jogo(s) inserido(s) com sucesso!")

In [ ]:
df_last = pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last["ganhou"] = np.where(df_last["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_last.columns = ["Concurso", "Data", "Bolas", "Ganhou?"]

df_last.tail(15)

In [ ]:
stmt = "SELECT * FROM megasena WHERE data IN (SELECT MAX(data) FROM megasena GROUP BY YEAR(data) HAVING YEAR(data) <> YEAR(CURRENT_DATE))"
df_mega_da_virada = pd.read_sql(sql=sa.text(stmt), con=engine)

df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada["ganhou"] = np.where(df_mega_da_virada["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_mega_da_virada.columns = ["Concurso", "Data", "Bolas", "Ganhou?"]

df_mega_da_virada